# Nested Composition Example

Replicating the nested composition example at https://algebraicjulia.github.io/SyntacticModels.jl/dev/generated/decapodes_examples/

Three models:
1. DragHeat (heat from friction)
2. NewtonCooling (heat dissipation)
3. LinearSuperposition (additive property of heat)

In [16]:
using Catlab
using Decapodes

In [2]:
import Pkg
Pkg.add("SyntacticModels")
using SyntacticModels.AMR
using SyntacticModels.ASKEMDecapodes
using SyntacticModels.ASKEMUWDs
using SyntacticModels.Composites

Pkg.add("JSON3")
using JSON3

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [14]:
sho = ASKEMDecaExpr(
    AMR.Header("harmonic_oscillator", "modelreps.io/DecaExpr", "A Simple Harmonic Oscillator as a Diagrammatic Equation", "DecaExpr", "v1.0"), 
    Decapodes.parse_decapode(quote
      X::Form0{Point}
      V::Form0{Point}

      k::Constant{Point}

      ∂ₜ(X) == V
      ∂ₜ(V) == -1*k*(X)
    end)
)

ASKEMDecaExpr(Header("harmonic_oscillator", "modelreps.io/DecaExpr", "A Simple Harmonic Oscillator as a Diagrammatic Equation", "DecaExpr", "v1.0"), Decapodes.DecaExpr(Judgement[Judgement(Decapodes.Var(:X), :Form0, :Point), Judgement(Decapodes.Var(:V), :Form0, :Point), Judgement(Decapodes.Var(:k), :Constant, :Point)], Decapodes.Equation[Eq(Tan(Decapodes.Var(:X)), Decapodes.Var(:V)), Eq(Tan(Decapodes.Var(:V)), Decapodes.Mult(Term[Decapodes.Lit(Symbol("-1")), Decapodes.Var(:k), Decapodes.Var(:X)]))]))

In [60]:
# Save to AMR JSON
open("sho.json", "w") do io 
  JSON3.pretty(io, sho, JSON3.AlignmentContext(indent = 2))
end

In [34]:
linear_drag = ASKEMDecaExpr(
  AMR.Header("LinearDragHeat", "modelreps.io/SummationDecapode", "Friction heat from linear or Stockes' drag", "SummationDecapode", "v1.0"),
  Decapodes.parse_decapode(quote
    V::Form0{Point}
    Q₊::Form0{Point}
    κ::Constant{Point}

    Q₊ == κ*V
  end)
)

ASKEMDecaExpr(Header("LinearDragHeat", "modelreps.io/SummationDecapode", "Friction heat from linear or Stockes' drag", "SummationDecapode", "v1.0"), Decapodes.DecaExpr(Judgement[Judgement(Decapodes.Var(:V), :Form0, :Point), Judgement(Decapodes.Var(:Q₊), :Form0, :Point), Judgement(Decapodes.Var(:κ), :Constant, :Point)], Decapodes.Equation[Eq(Decapodes.Var(:Q₊), App2(:*, Decapodes.Var(:κ), Decapodes.Var(:V)))]))

In [61]:
# Save to AMR JSON
open("linear_drag.json", "w") do io 
  JSON3.pretty(io, linear_drag, JSON3.AlignmentContext(indent = 2))
end

In [33]:
newton_cooling = ASKEMDecaExpr(
  AMR.Header("NetwonCooling", "modelreps.io/SummationDecapode", "Heat dissipation by Newton Cooling", "SummationDecapode", "v1.0"),
  Decapodes.parse_decapode(quote
    Q₋::Form0{Point}
    Q₀::Parameter{Point}
    Q::Form0{Point}
    λ::Constant{Point}

    Q₋ == λ(Q-Q₀)
  end)
)

ASKEMDecaExpr(Header("NetwonCooling", "modelreps.io/SummationDecapode", "Heat dissipation by Newton Cooling", "SummationDecapode", "v1.0"), Decapodes.DecaExpr(Judgement[Judgement(Decapodes.Var(:Q₋), :Form0, :Point), Judgement(Decapodes.Var(:Q₀), :Parameter, :Point), Judgement(Decapodes.Var(:Q), :Form0, :Point), Judgement(Decapodes.Var(:λ), :Constant, :Point)], Decapodes.Equation[Eq(Decapodes.Var(:Q₋), App1(:λ, App2(:-, Decapodes.Var(:Q), Decapodes.Var(:Q₀))))]))

In [62]:
# Save to AMR JSON
open("newton_cooling.json", "w") do io 
  JSON3.pretty(io, newton_cooling, JSON3.AlignmentContext(indent = 2))
end

In [35]:
linear_superposition = ASKEMDecaExpr(
  AMR.Header("LinearSuperpositon", "modelreps.io/SummationDecapode", "Additive property of heat", "SummationDecapode", "v1.0"),
  Decapodes.parse_decapode(quote
    X::Form0{Point}
    Y::Form0{Point}
    T::Form0{Point}

    T == X + Y
  end)
)

ASKEMDecaExpr(Header("LinearSuperpositon", "modelreps.io/SummationDecapode", "Additive property of heat", "SummationDecapode", "v1.0"), Decapodes.DecaExpr(Judgement[Judgement(Decapodes.Var(:X), :Form0, :Point), Judgement(Decapodes.Var(:Y), :Form0, :Point), Judgement(Decapodes.Var(:T), :Form0, :Point)], Decapodes.Equation[Eq(Decapodes.Var(:T), Plus(Term[Decapodes.Var(:X), Decapodes.Var(:Y)]))]))

In [63]:
# Save to AMR JSON
open("linear_superposition.json", "w") do io 
  JSON3.pretty(io, linear_superposition, JSON3.AlignmentContext(indent = 2))
end

## Define Composition Diagrams

In [48]:
x = Typed(:X, :Form0)
v = Typed(:V, :Form0)
Q = Typed(:Q, :Form0)
Q₊ = Untyped(:Q₊)
Q₋ = Untyped(:Q₋)
Q̇ = Untyped(:Q̇)

Untyped(:Q̇)

In [42]:
# Compose an oscillator model with a heat generation model

# oscillator_heating = @relation () begin
#   oscillator(x, v)
#   heatgen(v, Q)
# end

oscillator_heating_uwd = UWDExpr(
  [v, Q], 
  [Statement(:oscillator, [x, v]), Statement(:heatgen, [v, Q])]
)

UWDExpr(SyntacticModels.ASKEMUWDs.Var[Typed(:V, :Form0), Typed(:Q, :Form0)], Statement[Statement(:oscillator, SyntacticModels.ASKEMUWDs.Var[Typed(:X, :Form0), Typed(:V, :Form0)]), Statement(:heatgen, SyntacticModels.ASKEMUWDs.Var[Typed(:V, :Form0), Typed(:Q, :Form0)])])

In [39]:
# Compose a friction heating model from three component models

# drag_cooling = @relation () begin 
#   drag(v, Q₊)
#   cooling(Q₋, Q₊)
#   superposition(Q₊, Q₋, Q̇)
# end

drag_cooling_uwd = UWDExpr(
    [v, Q], 
    [Statement(:drag, [v, Q₊]), Statement(:cooling, [Q₋, Q]), Statement(:superposition, [Q₊, Q₋, Q̇])]
)

UWDExpr(SyntacticModels.ASKEMUWDs.Var[Typed(:V, :Form0), Typed(:Q, :Form0)], Statement[Statement(:drag, SyntacticModels.ASKEMUWDs.Var[Typed(:V, :Form0), Untyped(:Q₊)]), Statement(:cooling, SyntacticModels.ASKEMUWDs.Var[Untyped(:Q₋), Typed(:Q, :Form0)]), Statement(:superposition, SyntacticModels.ASKEMUWDs.Var[Untyped(:Q₊), Untyped(:Q₋), Untyped(:Q̇)])])

## First Composite Model

In [44]:
first_composite = CompositeModelExpr(
  AMR.Header("heating_dynamics", "modelreps.io/Composite", "A formula for heating-cooling", "CompositeModelExpr", "v0.1"),
  drag_cooling_uwd, 
  [
    OpenModel(linear_drag, [:V, :Q₊]), 
    OpenModel(newton_cooling, [:Q₋, :Q]), 
    OpenModel(linear_superposition, [:X, :Y, :T])
  ]
)   

CompositeModelExpr(Header("heating_dynamics", "modelreps.io/Composite", "A formula for heating - cooling", "CompositeModelExpr", "v0.1"), UWDExpr(SyntacticModels.ASKEMUWDs.Var[Typed(:V, :Form0), Typed(:Q, :Form0)], Statement[Statement(:drag, SyntacticModels.ASKEMUWDs.Var[Typed(:V, :Form0), Untyped(:Q₊)]), Statement(:cooling, SyntacticModels.ASKEMUWDs.Var[Untyped(:Q₋), Typed(:Q, :Form0)]), Statement(:superposition, SyntacticModels.ASKEMUWDs.Var[Untyped(:Q₊), Untyped(:Q₋), Untyped(:Q̇)])]), CompositeModel[OpenModel(ASKEMDecaExpr(Header("LinearDragHeat", "modelreps.io/SummationDecapode", "Friction heat from linear or Stockes' drag", "SummationDecapode", "v1.0"), Decapodes.DecaExpr(Judgement[Judgement(Decapodes.Var(:V), :Form0, :Point), Judgement(Decapodes.Var(:Q₊), :Form0, :Point), Judgement(Decapodes.Var(:κ), :Constant, :Point)], Decapodes.Equation[Eq(Decapodes.Var(:Q₊), App2(:*, Decapodes.Var(:κ), Decapodes.Var(:V)))])), [:V, :Q₊]), OpenModel(ASKEMDecaExpr(Header("NetwonCooling", "model

## Second Composite Model

In [46]:
second_composite = CompositeModelExpr(
  AMR.Header("hierarchical_composite", "modelreps.io/Composite", "A hierarchical composite model of frictional heating", "CompositeModelExpr", "v0.1"),
  oscillator_heating_uwd, 
  [
    OpenModel(sho, [:X, :V]), 
    first_composite
  ]
)

CompositeModelExpr(Header("hierarchical_composite", "modelreps.io/Composite", "A hierarchical composite model of frictional heating", "CompositeModelExpr", "v0.1"), UWDExpr(SyntacticModels.ASKEMUWDs.Var[Typed(:V, :Form0), Typed(:Q, :Form0)], Statement[Statement(:oscillator, SyntacticModels.ASKEMUWDs.Var[Typed(:X, :Form0), Typed(:V, :Form0)]), Statement(:heatgen, SyntacticModels.ASKEMUWDs.Var[Typed(:V, :Form0), Typed(:Q, :Form0)])]), CompositeModel[OpenModel(ASKEMDecaExpr(Header("harmonic_oscillator", "modelreps.io/DecaExpr", "A Simple Harmonic Oscillator as a Diagrammatic Equation", "DecaExpr", "v1.0"), Decapodes.DecaExpr(Judgement[Judgement(Decapodes.Var(:X), :Form0, :Point), Judgement(Decapodes.Var(:V), :Form0, :Point), Judgement(Decapodes.Var(:k), :Constant, :Point)], Decapodes.Equation[Eq(Tan(Decapodes.Var(:X)), Decapodes.Var(:V)), Eq(Tan(Decapodes.Var(:V)), Decapodes.Mult(Term[Decapodes.Lit(Symbol("-1")), Decapodes.Var(:k), Decapodes.Var(:X)]))])), [:X, :V]), CompositeModelExpr(He

## Flatten

In [70]:
# Flatten (but without metadata)
f = apex(oapply(second_composite))

Var,type,name
1,Form0,X
2,Form0,V
3,Constant,oscillator_k
4,infer,oscillator_mult_1
5,infer,oscillator_V̇
6,Literal,-1
7,Form0,heatgen_Q₊
8,Constant,heatgen_drag_κ
9,Form0,heatgen_Q₋
10,Parameter,heatgen_cooling_Q₀


In [66]:
# Flatten with AMR header
flat_composite_decapode = ASKEMDecapode(
    Header(
      "flattened_composite", 
      OpenDecapode(second_composite).model.header.schema, 
      "A flattened version of the composite_physics model.", 
      OpenDecapode(second_composite).model.header.schema_name, 
      OpenDecapode(second_composite).model.header.model_version
    ), 
    OpenDecapode(second_composite).model.model
)

ASKEMDecapode(Header("flattened_composite", "modelreps.io/Composite", "A flattened version of the composite_physics model.", "CompositeModelExpr", "v0.1"), SummationDecapode{Any, Any, Symbol}:
  Var = 1:14
  TVar = 1:2
  Op1 = 1:3
  Op2 = 1:4
  Σ = 1:1
  Summand = 1:2
  Type = 1:0
  Operator = 1:0
  Name = 1:0
  src : Op1 → Var = [1, 2, 13]
  tgt : Op1 → Var = [2, 5, 9]
  proj1 : Op2 → Var = [6, 4, 8, 11]
  proj2 : Op2 → Var = [3, 1, 2, 10]
  res : Op2 → Var = [4, 5, 7, 13]
  incl : TVar → Var = [2, 5]
  summand : Summand → Var = [7, 9]
  summation : Summand → Σ = [1, 1]
  sum : Σ → Var = [14]
  op1 : Op1 → Operator = [:∂ₜ, :∂ₜ, :λ]
  op2 : Op2 → Operator = [:*, :*, :*, :-]
  type : Var → Type = [:Form0, :Form0, :Constant, :infer, :infer, :Literal, :Form0, :Constant, :Form0, :Parameter, :Form0, :Constant, :infer, :Form0]
  name : Var → Name = [:X, :V, :oscillator_k, :oscillator_mult_1, :oscillator_V̇, Symbol("-1"), :heatgen_Q₊, :heatgen_drag_κ, :heatgen_Q₋, :heatgen_cooling_Q₀, :Q, :he

In [74]:
# Save to AMR JSON
open("flat_composite_model.json", "w") do io
    JSON3.pretty(io, flat_composite_decapode, JSON3.AlignmentContext(indent = 2))
end

LoadError: MethodError: no method matching _dict(::SummationDecapode{Any, Any, Symbol})

[0mClosest candidates are:
[0m  _dict([91m::Number[39m)
[0m[90m   @[39m [35mSyntacticModels[39m [90m~/.julia/packages/SyntacticModels/xH2xd/src/[39m[90m[4mSyntacticModelsBase.jl:24[24m[39m
[0m  _dict([91m::AbstractVector[39m)
[0m[90m   @[39m [35mSyntacticModels[39m [90m~/.julia/packages/SyntacticModels/xH2xd/src/[39m[90m[4mSyntacticModelsBase.jl:25[24m[39m
[0m  _dict([91m::T[39m) where T<:SyntacticModels.SyntacticModelsBase.AbstractTerm
[0m[90m   @[39m [35mSyntacticModels[39m [90m~/.julia/packages/SyntacticModels/xH2xd/src/[39m[90m[4mSyntacticModelsBase.jl:35[24m[39m
[0m  ...
